In [ ]:
import networkx as nx

attributes should be on nodes, not edges. parse/read_edgelist will place attributes on edges.
one file with edgelist, one file with nodes and their attributes. can combine these easily to create a networkx graph.

In [ ]:
# data = open('Datasets/hedden_network_with_followers_verified.csv', "r")
# next(data, None)  # skip the first line in the input file
# DiGraph = nx.DiGraph()

# G = nx.parse_edgelist(data, delimiter=',', create_using=DiGraph,
#                       nodetype=int, data=(('weight', float),))

in the meantime - use a dummy digraph

In [ ]:
import random

num_nodes = 1000
prob = 0.001
DummyGraph = nx.erdos_renyi_graph(n=num_nodes, p=prob, directed=True)
attr_dict = {}
for n in range(num_nodes):
    # get followers
    f = len(DummyGraph.pred[n])
    # simple weighted random assignment of verified status
    v = False
    if random.uniform(0,1) > 0.5:
        v = True
    attr_dict[n] = {"followers": f, "verified": v}

nx.set_node_attributes(DummyGraph, attr_dict)

# test
print(DummyGraph.nodes[0]["followers"])
print(DummyGraph.nodes[0]["verified"])
verified = nx.get_node_attributes(DummyGraph, "verified")
print("number of verified nodes:", sum(verified.values()))

nx.write_gexf(DummyGraph, "test.gexf")

Convert .csv network to .gexf file format

In [ ]:
import networkx as nx

data = open('cs4352-final-project/Scraping/dejan.csv', "r")
G = nx.DiGraph()

SoccerGraph = nx.parse_edgelist(data, delimiter=' ', create_using=G)
nx.write_gexf(SoccerGraph, "dejan_gephi.gexf")

Convert .gml network to .gexf file format

In [ ]:
SoccerGraph = nx.read_gml("Scraping/Dejan-Full-Node-Info.gml")
SoccerGraph = nx.convert_node_labels_to_integers(SoccerGraph, label_attribute="id")

nx.write_gexf(SoccerGraph, "dejan_full_node_info.gexf")

Inspect individual node attributes

In [ ]:
len(SoccerGraph.nodes())

## GCN

Convert networkx graph to dataset accessible by pytorch, cast all floats to integers (Longs)

In [ ]:
import torch
from torch_geometric.utils.convert import from_networkx

SoccerGraph = nx.read_gml("Scraping/Dejan-Full-Node-Info.gml")
SoccerGraph = nx.convert_node_labels_to_integers(SoccerGraph, label_attribute="id")

# rename verified to y attribute (verified) to graph before conversion 
for n, data in SoccerGraph.nodes(data=True):
    data["y"] = data.pop('verified')

data = from_networkx(SoccerGraph, group_node_attrs=['follower_count','friends_count','listed_count','favourites_count','statuses_count'])
print(data)

# equivalent of dataset.num_node_features
num_node_features = len(data.x[0])
num_classes = 2 # verified (y) is binary
print(num_node_features)

# TRYING TO FIX 'expected scalar type Long but found Float' error
# for i in range(len(data.x)):
#     data.x[i] = data.x[i].type(torch.FloatTensor)
    
# CRUCIAL
data.x = data.x.type(torch.FloatTensor)
    
    

Split up data into train and test

In [ ]:
# Split the data 
# ps 3 - train 140, val 500, test 1000
# pytorch docs: 'As a rule of thumb, we use 20% of the training set as the validation set.'

# Going with 60 training, 20 val, 20 test
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 1 - train_ratio - val_ratio

num_nodes = data.x.shape[0]
num_train = int(num_nodes * train_ratio)
num_val = int(num_nodes * val_ratio)
num_test = 1 - num_train - num_val
idx = [i for i in range(num_nodes)]

import numpy as np
np.random.shuffle(idx)

train_mask = torch.full_like(data.y, False, dtype=bool)
train_mask[idx[:num_train]] = True

val_mask = torch.full_like(data.y, False, dtype=bool)
val_mask[idx[num_train:num_train+num_val]] = True

test_mask = torch.full_like(data.y, False, dtype=bool)
test_mask[idx[num_train+num_val:]] = True

data['train_mask'] = train_mask
data['val_mask'] = val_mask
data['test_mask'] = test_mask



In [ ]:
count_true = 0
count_false = 0
for n in data.train_mask:
    if n:
        count_true += 1
    else:
        count_false += 1
print(count_true, count_false)
count_true = 0
count_false = 0
for n in data.val_mask:
    if n:
        count_true += 1
    else:
        count_false += 1
print(count_true, count_false)
count_true = 0
count_false = 0
for n in data.test_mask:
    if n:
        count_true += 1
    else:
        count_false += 1
print(count_true, count_false)

In [ ]:
# make sure train test val don't overlap
for i in range(num_nodes):
    if val_mask[i] and train_mask[i] or val_mask[i] and test_mask[i] or train_mask[i] and test_mask[i]:
        print('overlap')

In [ ]:
# remove unnecessary lists
data.__delattr__('name')
data.__delattr__('screen_name')
data.__delattr__('protected')
data.__delattr__('geo_enabled')
data.__delattr__('contributors_enabled')
data.__delattr__('is_translator')
data.__delattr__('is_translation_enabled')
data.__delattr__('profile_use_background_image')
data.__delattr__('has_extended_profile')
data.__delattr__('default_profile')
data.__delattr__('default_profile_image')
data.__delattr__('id')

print(data)

Define GCN

In [ ]:
# train partially on a general twitter network, and then finetune for each specific network?

import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Twitter_GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # two hidden layers
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = data.to(device)

# data.validate(raise_on_error=True)

model = Twitter_GCN().to(device) 

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


train, test

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

evaluation

In [ ]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

# FROM DATASET IN PROB 3 LOOKS LIKE 20% IS USED TO TRAIN AND 80% TO TEST INSTEAD OF VICE VERSA -- 
# IS THIS WHY ACCURACY IS SO HIGH??


change to val_mask and tune hyperparameters just like hw 3

In [ ]:
import statistics

class GCN(torch.nn.Module):
    def __init__(self, num_layers):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(num_node_features, 16))
        for i in range(num_layers):
            self.layers.append(GCNConv(16, 16))
        self.layers.append(GCNConv(16, num_classes))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        for layer in self.layers[:-1]:
            x = layer(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
        x = self.layers[-1](x, edge_index)

        return F.log_softmax(x, dim=1)


num_layers = [2, 10]
l2_regs = [0, 0.0001, 0.00000000001]
learning_rates = [0.9, 0.01, 0.001]

NUM_ITERS = 2

all_trials = []
for n in num_layers:            
    for l2_reg in l2_regs:
        for learning_rate in learning_rates:
            
            results = []
            
            for i in range(NUM_ITERS):
                model = Twitter_GCN().to(device)
                
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg)

                model.train()
                for epoch in range(200):
                    optimizer.zero_grad()
                    out = model(data)
                    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
                    loss.backward()
                    optimizer.step()
                model.eval()
                pred = model(data).argmax(dim=1)
                correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
                acc = int(correct) / int(data.val_mask.sum())
                print(n, l2_reg, learning_rate)
                print(f'Accuracy: {acc:.4f}')
                
                results.append(acc)
                
            avg = sum(results) / len(results)
            stdev = statistics.pstdev(results)
            all_trials.append(((n, l2_reg, learning_rate), avg, stdev))

print(("Number of layers", "L2 regularization", "Learning rate"), "Average", "Standard Dev.")
for t in all_trials:
    print(t)

max = max(all_trials, key=lambda x:x[1])
print(max)

linear regression model as a baseline

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression


model = LinearRegression().fit(data.x, data.y)

r_sq = model.score(data.x, data.y)
print(f"coefficient of determination: {r_sq}")

do neural net without graph info?

train a model on each network

In [14]:
import networkx as nx
import torch
from torch_geometric.utils.convert import from_networkx
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


def mask_data(data, train_ratio, val_ratio):

    test_ratio = 1 - train_ratio - val_ratio

    num_nodes = data.x.shape[0]
    num_train = int(num_nodes * train_ratio)
    num_val = int(num_nodes * val_ratio)
    num_test = 1 - num_train - num_val
    idx = [i for i in range(num_nodes)]

    np.random.shuffle(idx)

    train_mask = torch.full_like(data.y, False, dtype=bool)
    train_mask[idx[:num_train]] = True

    val_mask = torch.full_like(data.y, False, dtype=bool)
    val_mask[idx[num_train:num_train+num_val]] = True

    test_mask = torch.full_like(data.y, False, dtype=bool)
    test_mask[idx[num_train+num_val:]] = True

    data['train_mask'] = train_mask
    data['val_mask'] = val_mask
    data['test_mask'] = test_mask
    

class Twitter_GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()
        # two hidden layers
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
 


def GCN_on_Twitter_Graph(gml_file):
    SoccerGraph = nx.read_gml(gml_file)
    SoccerGraph = nx.convert_node_labels_to_integers(SoccerGraph, label_attribute="id")

    # rename verified to y attribute (verified) to graph before conversion 
    for n, data in SoccerGraph.nodes(data=True):
        data["y"] = data.pop('verified')

    data = from_networkx(SoccerGraph, group_node_attrs=['follower_count','friends_count','listed_count','favourites_count','statuses_count'])
#     data = from_networkx(SoccerGraph, group_node_attrs=['followers'])

    num_node_features = len(data.x[0])
    num_classes = 2

    data.x = data.x.type(torch.FloatTensor)
    
    train_ratio = 0.4
    val_ratio = 0.3
    mask_data(data, train_ratio, val_ratio)
    
#     print(data)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    data = data.to(device)

    data.validate(raise_on_error=True)

    model = Twitter_GCN(num_node_features,num_classes).to(device) 

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
    # train on own model
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    print(f'Accuracy: {acc:.4f}')
    
    
print('dejan')
GCN_on_Twitter_Graph('Scraping/Dejan-Full-Node-Info.gml')
print('mccormick')
GCN_on_Twitter_Graph('Scraping/McCormick.gml')
print('hedden')
GCN_on_Twitter_Graph('Scraping/Hedden.gml')
# print('FAKE')
# GCN_on_Twitter_Graph('fakeNetwork.gml')


dejan
Accuracy: 0.9757
mccormick
Accuracy: 0.9009
hedden
Accuracy: 0.8136


test each model on the other networks

In [19]:
train_ratio = 0.4
val_ratio = 0.3

DejanGraph = nx.read_gml('Scraping/Dejan-Full-Node-Info.gml')
DejanGraph = nx.convert_node_labels_to_integers(DejanGraph, label_attribute="id")

# rename verified to y attribute (verified) to graph before conversion 
for n, data in DejanGraph.nodes(data=True):
    data["y"] = data.pop('verified')

DejanData = from_networkx(DejanGraph, group_node_attrs=['follower_count','friends_count','listed_count','favourites_count','statuses_count'])
#     data = from_networkx(SoccerGraph, group_node_attrs=['followers'])

num_node_features = len(DejanData.x[0])
num_classes = 2

DejanData.x = DejanData.x.type(torch.FloatTensor)


McCormickGraph = nx.read_gml('Scraping/McCormick.gml')
McCormickGraph = nx.convert_node_labels_to_integers(McCormickGraph, label_attribute="id")

# rename verified to y attribute (verified) to graph before conversion 
for n, data in McCormickGraph.nodes(data=True):
    data["y"] = data.pop('verified')

McCormickData = from_networkx(McCormickGraph, group_node_attrs=['follower_count','friends_count','listed_count','favourites_count','statuses_count'])
#     data = from_networkx(SoccerGraph, group_node_attrs=['followers'])

num_node_features = len(McCormickData.x[0])
num_classes = 2

McCormickData.x = McCormickData.x.type(torch.FloatTensor)


HeddenGraph = nx.read_gml('Scraping/Hedden.gml')
HeddenGraph = nx.convert_node_labels_to_integers(HeddenGraph, label_attribute="id")

# rename verified to y attribute (verified) to graph before conversion 
for n, data in HeddenGraph.nodes(data=True):
    data["y"] = data.pop('verified')

HeddenData = from_networkx(HeddenGraph, group_node_attrs=['follower_count','friends_count','listed_count','favourites_count','statuses_count'])

num_node_features = len(HeddenData.x[0])
num_classes = 2

HeddenData.x = HeddenData.x.type(torch.FloatTensor)



# define method to train on one graph, test on the other two
def test_on_other_two(train_data, test_data1, test_data2):
    
    mask_data(train_data, train_ratio=0.6, val_ratio=0.2)
    mask_data(test_data1, train_ratio=0, val_ratio=0)
    mask_data(test_data2, train_ratio=0, val_ratio=0)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data = train_data.to(device)
    test_data1 = test_data1.to(device)
    test_data2 = test_data2.to(device)

    train_data.validate(raise_on_error=True)
    test_data1.validate(raise_on_error=True)
    test_data2.validate(raise_on_error=True)

    model = Twitter_GCN(num_node_features,num_classes).to(device) 

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    # train on one model
    model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(train_data)
        loss = F.nll_loss(out[train_data.train_mask], train_data.y[train_data.train_mask])
        loss.backward()
        optimizer.step()
        
        
    # test on trained on model
    model.eval()
    pred = model(train_data).argmax(dim=1)
    correct = (pred[train_data.test_mask] == train_data.y[train_data.test_mask]).sum()
    acc = int(correct) / int(train_data.test_mask.sum())
    print(f'Accuracy on trained on model: {acc:.4f}')    
        
    # test on other two
    model.eval()
    pred = model(test_data1).argmax(dim=1)
    correct = (pred[test_data1.test_mask] == test_data1.y[test_data1.test_mask]).sum()
    acc = int(correct) / int(test_data1.test_mask.sum())
    print(f'Accuracy on other model: {acc:.4f}')
    
    model.eval()
    pred = model(test_data2).argmax(dim=1)
    correct = (pred[test_data2.test_mask] == test_data2.y[test_data2.test_mask]).sum()
    acc = int(correct) / int(test_data2.test_mask.sum())
    print(f'Accuracy on other other model: {acc:.4f}')

 
print("trained on dejan_________")
print("Dejan, McCormick, Hedden")
test_on_other_two(DejanData, McCormickData, HeddenData)

 
print("trained on mccormick_________")
print("McCormick, Dejan, Hedden")
test_on_other_two(McCormickData, DejanData, HeddenData)

 
print("trained on hedden_________")
print("Hedden, Dejan, McCormick")
test_on_other_two(HeddenData, DejanData, McCormickData)


trained on dejan_________
Dejan, McCormick, Hedden
Accuracy on trained on model: 0.9854
Accuracy on other model: 0.8765
Accuracy on other other model: 0.8201
trained on mccormick_________
McCormick, Dejan, Hedden
Accuracy on trained on model: 0.7206
Accuracy on other model: 0.6161
Accuracy on other other model: 0.7253
trained on hedden_________
Hedden, Dejan, McCormick
Accuracy on trained on model: 0.9143
Accuracy on other model: 0.9869
Accuracy on other other model: 0.8814


if i have time: try out a neural network w/o graph data -- is that just as accurate?